# Example: Use duplicated symbol to find token id

Use DOGE as an example, DOGE is widely use by shitcoin. So there would be tons of duplicated token symbol. But we can shortlist their token id through cg.

In [46]:
import requests
import pandas as pd
import os
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()

def get_token_ids_by_symbol(symbol, max_ids=None):
    print(f"开始获取符号 '{symbol}' 的 token IDs...")
    url = "https://pro-api.coingecko.com/api/v3/coins/markets"
    token_ids = []
    page = 1
    api_key = os.getenv('COINGECKO_API_KEY')

    while True:
        print(f"正在请求第 {page} 页数据...")
        params = {
            'vs_currency': 'usd',
            'order': 'id_asc',
            'per_page': 100,
            'page': page,
            'sparkline': False
        }
        headers = {
            'Accepts': 'application/json',
            'X-CG-PRO-API-KEY': api_key
        }

        response = requests.get(url, headers=headers, params=params)

        if response.status_code == 200:
            data = response.json()
            if not data:
                print("未找到更多数据，结束请求。")
                break

            for token in data:
                if token['symbol'].lower() == symbol.lower():
                    token_ids.append(token['id'])
                    if max_ids is not None and len(token_ids) >= max_ids:
                        print(f"已找到 {len(token_ids)} 个 token IDs，结束请求。")
                        return token_ids

            if len(data) < 100:
                print("返回的数据少于 100，说明没有更多数据，结束请求。")
                break

            page += 1
        else:
            print(f"获取数据时出错: {response.status_code}")
            break

    print(f"共找到 {len(token_ids)} 个 token IDs。")
    return token_ids

def get_market_data(token_id):
    print(f"正在获取 token ID '{token_id}' 的市场数据...")
    url = f"https://pro-api.coingecko.com/api/v3/coins/{token_id}/market_chart"
    params = {
        'vs_currency': 'usd',
        'days': '30',
        'interval': 'daily'
    }
    headers = {
        'Accepts': 'application/json',
        'X-CG-PRO-API-KEY': os.getenv('COINGECKO_API_KEY')
    }

    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        print(f"成功获取 token ID '{token_id}' 的市场数据。")
        return response.json()
    else:
        print(f"获取市场数据时出错: {response.status_code}")
        return None

def save_to_csv(token_id):
    market_data = get_market_data(token_id)
    if market_data is None:
        print(f"无法获取 token ID '{token_id}' 的市场数据，结束。")
        return
    
    print(f"正在保存 token ID '{token_id}' 的市场数据到 CSV 文件...")
    prices = market_data['prices']
    market_caps = market_data['market_caps']
    total_volumes = market_data['total_volumes']

    data = {
        'date': [],
        'price': [],
        'market_cap': [],
        'volume': []
    }

    for i in range(len(prices)):
        data['date'].append(pd.to_datetime(prices[i][0], unit='ms'))
        data['price'].append(prices[i][1])
        data['market_cap'].append(market_caps[i][1])
        data['volume'].append(total_volumes[i][1])

    df = pd.DataFrame(data)

    # 获取 token 的基本信息
    token_info = requests.get(f"https://pro-api.coingecko.com/api/v3/coins/{token_id}", 
                               headers={'X-CG-PRO-API-KEY': os.getenv('COINGECKO_API_KEY')}).json()
    name = token_info.get('name', 'Unknown')
    symbol = token_info.get('symbol', 'unknown')

    df['id'] = token_id
    df['symbol'] = symbol
    df['name'] = name

    os.makedirs('cg_token', exist_ok=True)

    # 修改 CSV 文件名，确保唯一性
    csv_file_path = f"cg_token/{symbol}_{token_id}.csv"
    df.to_csv(csv_file_path, index=False)
    print(f"数据已保存到 {csv_file_path}。")

# 示例用法
symbol = "DOGE"  # 替换为您想要查找的 symbol
max_ids = 3  # 设置为数字，或使用 None 获取所有可用的 ID
token_ids = get_token_ids_by_symbol(symbol, max_ids)

# 根据获取的 token IDs 获取市场数据并保存到 CSV
for token_id in token_ids:
    save_to_csv(token_id)
    
print(token_id)

开始获取符号 'DOGE' 的 token IDs...
正在请求第 1 页数据...
正在请求第 2 页数据...
正在请求第 3 页数据...
正在请求第 4 页数据...
正在请求第 5 页数据...
正在请求第 6 页数据...
正在请求第 7 页数据...
正在请求第 8 页数据...
正在请求第 9 页数据...
正在请求第 10 页数据...
正在请求第 11 页数据...
正在请求第 12 页数据...
正在请求第 13 页数据...
正在请求第 14 页数据...
正在请求第 15 页数据...
正在请求第 16 页数据...
正在请求第 17 页数据...
正在请求第 18 页数据...
正在请求第 19 页数据...
正在请求第 20 页数据...
正在请求第 21 页数据...
正在请求第 22 页数据...
正在请求第 23 页数据...
正在请求第 24 页数据...
正在请求第 25 页数据...
正在请求第 26 页数据...
正在请求第 27 页数据...
正在请求第 28 页数据...
正在请求第 29 页数据...
正在请求第 30 页数据...
正在请求第 31 页数据...
正在请求第 32 页数据...
正在请求第 33 页数据...
正在请求第 34 页数据...
正在请求第 35 页数据...
正在请求第 36 页数据...
正在请求第 37 页数据...
正在请求第 38 页数据...
正在请求第 39 页数据...
正在请求第 40 页数据...
正在请求第 41 页数据...
已找到 3 个 token IDs，结束请求。
正在获取 token ID 'binance-peg-dogecoin' 的市场数据...
成功获取 token ID 'binance-peg-dogecoin' 的市场数据。
正在保存 token ID 'binance-peg-dogecoin' 的市场数据到 CSV 文件...
数据已保存到 cg_token/doge_binance-peg-dogecoin.csv。
正在获取 token ID 'department-of-gov-efficiency' 的市场数据...
成功获取 token ID 'department-of-gov-efficiency' 的市场数据。
正在保

# CCXT data Prepartion



In [47]:
import ccxt
import pandas as pd
import os

def fetch_doge_pairs(exchange):
    # 获取所有市场
    markets = exchange.load_markets()
    # 筛选出与 DOGE 和 USDT 相关的交易对
    doge_pairs = [symbol for symbol in markets if 'doge' in symbol.lower()]
    return doge_pairs

def fetch_ohlcv(exchange, pair):
    # 获取OHLCV数据，时间单位为一天，限制为最近30天
    ohlcv = exchange.fetch_ohlcv(pair, timeframe='1d', limit=30)
    return ohlcv

def save_ohlcv_to_csv(pair, ohlcv_data):
    # 创建文件夹
    folder = "ccxt_token"
    os.makedirs(folder, exist_ok=True)

    # 将 OHLCV 数据保存为 CSV 文件
    df = pd.DataFrame(ohlcv_data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')  # 转换时间戳
    file_path = os.path.join(folder, f"{pair.replace('/', '_')}.csv")
    df.to_csv(file_path, index=False)
    print(f"Saved OHLCV data for {pair} to {file_path}")

# 创建 MEXC 交易所实例
exchange = ccxt.mexc()

# 查找所有与 DOGE 相关的 USDT 交易对
doge_pairs = fetch_doge_pairs(exchange)

# 获取每个交易对的 OHLCV 数据并保存
for pair in doge_pairs:
    ohlcv_data = fetch_ohlcv(exchange, pair)
    if ohlcv_data:
        save_ohlcv_to_csv(pair, ohlcv_data)

Saved OHLCV data for DOGEVERSE/USDT to ccxt_token/DOGEVERSE_USDT.csv
Saved OHLCV data for DOGELON/USDT to ccxt_token/DOGELON_USDT.csv
Saved OHLCV data for DOGEGOV/USDT to ccxt_token/DOGEGOV_USDT.csv
Saved OHLCV data for DOGE2/USDT to ccxt_token/DOGE2_USDT.csv
Saved OHLCV data for DOGE/USDT to ccxt_token/DOGE_USDT.csv
Saved OHLCV data for DOGEETH/USDT to ccxt_token/DOGEETH_USDT.csv
Saved OHLCV data for DOGE/USDC to ccxt_token/DOGE_USDC.csv
Saved OHLCV data for AIDOGEXLM/USDT to ccxt_token/AIDOGEXLM_USDT.csv
Saved OHLCV data for BABYDOGE2/USDT to ccxt_token/BABYDOGE2_USDT.csv
Saved OHLCV data for DOGECOIN/USDT to ccxt_token/DOGECOIN_USDT.csv
Saved OHLCV data for AIDOGE/USDT to ccxt_token/AIDOGE_USDT.csv
Saved OHLCV data for ELONDOGE/USDT to ccxt_token/ELONDOGE_USDT.csv
Saved OHLCV data for DOGE20/USDT to ccxt_token/DOGE20_USDT.csv
Saved OHLCV data for POLYDOGE/USDT to ccxt_token/POLYDOGE_USDT.csv
Saved OHLCV data for DOGEX/USDT to ccxt_token/DOGEX_USDT.csv
Saved OHLCV data for DOGECEO/US

# check DOGE symbol 

There are more than 10 repeated symbol for DOGE, if we only use top 3 by id_asc. Then the token id result is following

`'binance-peg-dogecoin','department-of-gov-efficiency','department-of-government-efficiency'`

How can we know these token could be matched with ccxt token symbol? --- By date price difference.

Criteria :

1. 5% price deviation --> matched 

2. Under last 30d window 

3. If there is >= 50% matched symbol exists, then the cg token should belong to that symbol on ccxt


In [141]:
import pandas as pd
import os

def load_cg_token_data(folder):
    cg_data = []
    for file in os.listdir(folder):
        if file.endswith('.csv'):
            try:
                df = pd.read_csv(os.path.join(folder, file))
                # 将 'date' 列转换为 datetime 格式
                df['date'] = pd.to_datetime(df['date'])
                # 只保留时间为 00:00:00 的行
                df = df[df['date'].dt.time == pd.to_datetime('00:00:00').time()]
                cg_data.append(df)
            except Exception as e:
                print(f"Error loading {file}: {e}")
    return pd.concat(cg_data, ignore_index=True)

def load_ccxt_token_data(folder):
    ccxt_data = {}
    for file in os.listdir(folder):
        if file.endswith('.csv'):
            symbol = file.replace('.csv', '').replace('_', '/')
            try:
                df = pd.read_csv(os.path.join(folder, file))
                ccxt_data[symbol] = df
            except Exception as e:
                print(f"Error loading {file}: {e}")
    return ccxt_data  # 确保返回 ccxt_data

def find_matching_tokens(cg_df, ccxt_data, output_folder):
    # 确保 'date' 列为日期格式
    cg_df['date'] = pd.to_datetime(cg_df['date'])

    # 逐行处理每个 cg token
    grouped_cg = cg_df.groupby('id')

    for cg_id, group in grouped_cg:
        # 获取 token 的所有行
        all_dates = group['date'].dt.date.unique()
        token_name = group['name'].iloc[0]  # 获取 token 名称
        token_symbol = group['symbol'].iloc[0]  # 获取 token symbol
        matched_symbols = []

        # 在 ccxt token 中查找匹配的价格
        for ccxt_symbol, ccxt_df in ccxt_data.items():
            if 'open' in ccxt_df.columns and 'timestamp' in ccxt_df.columns:
                ccxt_df['timestamp'] = pd.to_datetime(ccxt_df['timestamp'])
                
                # 检查每一个日期
                for date in all_dates:
                    matching_rows = ccxt_df[ccxt_df['timestamp'].dt.date == date]
                    if not matching_rows.empty:
                        ccxt_price = matching_rows['open'].iloc[-1]  # 获取该日期的 close price
                        cg_row = group[group['date'].dt.date == date]
                        
                        if not cg_row.empty:
                            cg_price = cg_row['price'].iloc[0]  # 获取该日期的 price
                            # 计算上界和下界
                            upper_bound = cg_price * 1.05
                            lower_bound = cg_price * 0.95
                            # 检查 ccxt_price 是否在上下界之间
                            if lower_bound <= ccxt_price <= upper_bound:
                                matched_symbols.append((date, ccxt_symbol))  # 记录

        # 将匹配结果添加到结果列表
        results = []
        for date in all_dates:
            match_symbol = ', '.join([symbol for d, symbol in matched_symbols if d == date]) or 'None'
            results.append({
                'date': date,
                'cg_price': group[group['date'].dt.date == date]['price'].iloc[0] if not group[group['date'].dt.date == date].empty else None,
                'name': token_name,
                'id': cg_id,
                'symbol': token_symbol,
                'matched_ccxt_symbols': match_symbol
            })

        # 创建独立的 CSV 文件
        output_file = os.path.join(output_folder, f"{cg_id}_matches.csv")
        pd.DataFrame(results).to_csv(output_file, index=False)
        print(f"Saved match results for {token_symbol} to {output_file}")

def main():
    cg_folder = 'cg_token'  # cg_token 文件夹路径
    ccxt_folder = 'ccxt_token'  # ccxt_token 文件夹路径
    output_folder = 'paired_token'  # 输出文件夹路径
    os.makedirs(output_folder, exist_ok=True)

    # 加载数据
    cg_df = load_cg_token_data(cg_folder)
    ccxt_data = load_ccxt_token_data(ccxt_folder)

    # 查找匹配的 tokens
    find_matching_tokens(cg_df, ccxt_data, output_folder)

if __name__ == "__main__":
    main()

Saved match results for doge to paired_token/binance-peg-dogecoin_matches.csv
Saved match results for doge to paired_token/department-of-gov-efficiency_matches.csv
Saved match results for doge to paired_token/department-of-government-efficiency_matches.csv


In [149]:
def classify_tokens_by_symbol(output_folder):
    classification_results = []
    
    for file in os.listdir(output_folder):
        if file.endswith('.csv') and file != 'classification_results.csv':
            df = pd.read_csv(os.path.join(output_folder, file))
            
            # 检查列名
            if 'matched_ccxt_symbols' not in df.columns:
                print(f"Warning: 'matched_ccxt_symbols' not found in {file}. Columns available: {df.columns.tolist()}")
                continue

            symbol_counts = {}
            for matched_symbols in df['matched_ccxt_symbols']:
                # 处理缺失值
                if pd.isna(matched_symbols):
                    continue
                
                # 确保 matched_symbols 是字符串
                if isinstance(matched_symbols, str):
                    symbols = [symbol.strip() for symbol in matched_symbols.split(',')]
                    for symbol in symbols:
                        if symbol != 'None':
                            if symbol not in symbol_counts:
                                symbol_counts[symbol] = 0
                            symbol_counts[symbol] += 1
                else:
                    print(f"Warning: Unexpected type for matched_symbols in {file}: {type(matched_symbols)}")
                    
            for symbol, count in symbol_counts.items():
                if count > 15:
                    classification_results.append({
                        'token_id': df['id'].iloc[0],
                        'token_name': df['name'].iloc[0],
                        'classified_symbol': symbol
                    })
    
    classification_df = pd.DataFrame(classification_results)
    classification_output_file = os.path.join(output_folder, 'classification_results.csv')
    classification_df.to_csv(classification_output_file, index=False)
    print(f"Saved classification results to {classification_output_file}")

def main():
    cg_folder = 'cg_token'
    ccxt_folder = 'ccxt_token'
    output_folder = 'paired_token'
    os.makedirs(output_folder, exist_ok=True)

    cg_df = load_cg_token_data(cg_folder)
    ccxt_data = load_ccxt_token_data(ccxt_folder)

    find_matching_tokens(cg_df, ccxt_data, output_folder)
    classify_tokens_by_symbol(output_folder)

if __name__ == "__main__":
    main()

Saved match results for doge to paired_token/binance-peg-dogecoin_matches.csv
Saved match results for doge to paired_token/department-of-gov-efficiency_matches.csv
Saved match results for doge to paired_token/department-of-government-efficiency_matches.csv
Saved classification results to paired_token/classification_results.csv


# test code

In [134]:
import pandas as pd
import os

def load_cg_token_data(filepath):
    try:
        df = pd.read_csv(filepath)
        df['date'] = pd.to_datetime(df['date'])
        df = df[df['date'].dt.time == pd.to_datetime('00:00:00').time()]
        #print(f"Loaded CG data from {filepath}: {df.head()}")
        return df
    except Exception as e:
        print(f"Error loading {filepath}: {e}")
        return pd.DataFrame()  # 返回空 DataFrame

def load_ccxt_token_data(filepath):
    try:
        df = pd.read_csv(filepath)
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        #print(f"Loaded CCXT data from {filepath}: {df.head()}")
        return df
    except Exception as e:
        print(f"Error loading {filepath}: {e}")
        return pd.DataFrame()  # 返回空 DataFrame

def find_matching_tokens(cg_df, ccxt_df):
    cg_df['date'] = pd.to_datetime(cg_df['date'])
    all_dates = cg_df['date'].dt.date.unique()
    
    matched_symbols = []

    for date in all_dates:
        matching_rows = ccxt_df[ccxt_df['timestamp'].dt.date == date]
        if not matching_rows.empty:
            ccxt_price = matching_rows['open'].iloc[-1]
            cg_row = cg_df[cg_df['date'].dt.date == date]
            

            if not cg_row.empty:
                cg_price = cg_row['price'].iloc[0]
                upper_bound = cg_price * 1.05
                lower_bound = cg_price * 0.95

                print(f"Date: {date}, CG Price: {cg_price}, CCXT Price: {ccxt_price}, "
                      f"Lower Bound: {lower_bound}, Upper Bound: {upper_bound}")

                if lower_bound <= ccxt_price <= upper_bound:
                    matched_symbols.append((date, ccxt_price))
                    print(f"Match found: {date} -> CCXT Price: {ccxt_price}")

    if matched_symbols:
        print(f"Matched symbols:")
        for match in matched_symbols:
            print(f"  Date: {match[0]}, CCXT Price: {match[1]}")
    else:
        print("No matches found.")

def main():
    cg_file = 'cg_token/doge_department-of-government-efficiency.csv'  # 指定文件路径
    ccxt_file = 'ccxt_token/DOGEGOV_USDT.csv'  # 指定文件路径

    # 加载指定的 CG 数据
    cg_df = load_cg_token_data(cg_file)
    # 加载指定的 CCXT 数据
    ccxt_df = load_ccxt_token_data(ccxt_file)

    # 查找匹配的 tokens
    find_matching_tokens(cg_df, ccxt_df)

if __name__ == "__main__":
    main()

Date: 2024-11-20, CG Price: 0.2239063890508507, CCXT Price: 0.1, Lower Bound: 0.21271106959830816, Upper Bound: 0.23510170850339324
Date: 2024-11-21, CG Price: 0.2304956954934472, CCXT Price: 0.2333, Lower Bound: 0.21897091071877484, Upper Bound: 0.24202048026811956
Match found: 2024-11-21 -> CCXT Price: 0.2333
Date: 2024-11-22, CG Price: 0.314444043183829, CCXT Price: 0.3107, Lower Bound: 0.29872184102463756, Upper Bound: 0.33016624534302047
Match found: 2024-11-22 -> CCXT Price: 0.3107
Date: 2024-11-23, CG Price: 0.2581137064490421, CCXT Price: 0.2583, Lower Bound: 0.24520802112658996, Upper Bound: 0.2710193917714942
Match found: 2024-11-23 -> CCXT Price: 0.2583
Date: 2024-11-24, CG Price: 0.2311695487950237, CCXT Price: 0.2315, Lower Bound: 0.2196110713552725, Upper Bound: 0.24272802623477488
Match found: 2024-11-24 -> CCXT Price: 0.2315
Date: 2024-11-25, CG Price: 0.2269053927574893, CCXT Price: 0.2238, Lower Bound: 0.21556012311961484, Upper Bound: 0.23825066239536377
Match found: